In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [202]:
import datetime as dt

In [203]:
df = pd.read_csv('./prescription_merged_final.csv',
                 index_col=0,
                 parse_dates=['date', 'birthdate']
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60800 entries, 0 to 60799
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            60800 non-null  datetime64[ns]
 1   member          60800 non-null  int64         
 2   service         60800 non-null  int64         
 3   quantity        60800 non-null  int64         
 4   deduction       60800 non-null  int64         
 5   insurance_paid  60800 non-null  int64         
 6   patient_paid    60800 non-null  int64         
 7   province        60800 non-null  int64         
 8   prescription    60800 non-null  int64         
 9   specialty_en    59717 non-null  object        
 10  birthdate       60800 non-null  datetime64[ns]
 11  gender          60800 non-null  int64         
 12  atc             60800 non-null  object        
 13  dm              60800 non-null  bool          
 14  ab              60800 non-null  bool          
 15  calc_dd

In [ ]:
df

## Calculating days of usage

In [ ]:
ab_df = df.loc[df['ab']].copy()
ab_df

Creating ab_duration as the duration of prescribed antibiotic in days.

In [206]:
ab_df.loc[:, 'ab_duration'] = pd.to_timedelta(
    ab_df['quantity'] * ab_df['calc_ddd'], unit='day'
)

In [ ]:
ab_df

Creating date_end as the end of the prescribed antibiotic duration.

In [ ]:
ab_df.loc[:, 'end_date'] = ab_df['date'] + ab_df['ab_duration']
ab_df

Grouping by member and sorting by date.\
The groupby is probably not useful.

In [209]:
ab_df_date = ab_df[['member', 'prescription', 'date', 'end_date']].copy()

In [ ]:
ab_df_date.sort_values(by=['member', 'date'], inplace=True, ignore_index=True)
ab_df_date

In [ ]:
ab_df_date['is_same'] = ab_df_date['member'] == ab_df_date.shift(1)['member']
ab_df_date

In [268]:
combine_period = 90
combined_duration = {m: [] for m in ab_df_date['member'].unique()}
combined_duration['first'] = []
duration = dt.timedelta(days=0)
start = None
end = None
member = 'first'
for i, row in ab_df_date.iterrows():

    if not row['is_same']:
        # Saving duration for previous member
        combined_duration[member].append((i, duration))
        # Setting variables for new member
        member = row['member']
        start = row['date']
        end = row['end_date']
        duration = end - start
    elif (row['date'] - end) <= dt.timedelta(days=combine_period):
        # Setting the new end for period
        if row['end_date'] > end:
            end = row['end_date']
            duration = end - start
    else:
        # Setting start and end for new reference point
        start = row['date']
        end = row['end_date']
        # Saving duration as the period has ended
        combined_duration[member].append((i, duration))

Some durations might be duplicates I think. Don't rely on them. You rely on the min and max of the durations though. The number of duration duplicates are low if any.

In [269]:
# Removing i from combined durations
simple_durations = dict()
for member, ls_d in combined_duration.items():
    new_ls_d = []
    for i, duration in ls_d:
        new_ls_d.append(duration)
    simple_durations[member] = new_ls_d

In [270]:
# Saving the chronic members to a list
chronic_period = 90
chronic_members = set()
for member, ls_d in simple_durations.items():
    for duration in ls_d:
        if duration >= dt.timedelta(days=chronic_period):
            chronic_members.add(member)
len(chronic_members)

312

I now realized that I could have done more of the work by vectorization. I could use a window period 0f like 90 days and if in any day there was AB prescribed, I would have set a column to true.